In [21]:
# import reframed
import cobra
import pandas as pd
import numpy as np
from pathlib import Path
import scipy.stats as st


# Analyse fluxes, shadow prices, growth and changes in intracellular concentrations to choose KO strains

In [3]:
model = cobra.io.read_sbml_model('../../models/e_coli/momentiJO1366_modified.xml')


Set parameter Username
Set parameter LicenseID to value 2586336
Academic license - for non-commercial use only - expires 2025-11-18


Adding exchange reaction EX_12ppd__R_e with default bounds for boundary metabolite: 12ppd__R_e.
Adding exchange reaction EX_12ppd__S_e with default bounds for boundary metabolite: 12ppd__S_e.
Adding exchange reaction EX_14glucan_e with default bounds for boundary metabolite: 14glucan_e.
Adding exchange reaction EX_15dap_e with default bounds for boundary metabolite: 15dap_e.
Adding exchange reaction EX_23camp_e with default bounds for boundary metabolite: 23camp_e.
Adding exchange reaction EX_23ccmp_e with default bounds for boundary metabolite: 23ccmp_e.
Adding exchange reaction EX_23cgmp_e with default bounds for boundary metabolite: 23cgmp_e.
Adding exchange reaction EX_23cump_e with default bounds for boundary metabolite: 23cump_e.
Adding exchange reaction EX_23dappa_e with default bounds for boundary metabolite: 23dappa_e.
Adding exchange reaction EX_26dap__M_e with default bounds for boundary metabolite: 26dap__M_e.
Adding exchange reaction EX_2ddglcn_e with default bounds for bo

### Knockout gene b4301 because it is seems to be a wrongly annotated homolog of rpe

In [4]:
model.genes.get_by_id('b4301').knock_out() #This seems to be wrongly annotated rpe homolog

# Set M9 galactose environment

In [17]:
# M9
uptake = ['EX_nh4_e', 'EX_pi_e', 'EX_so4_e', 'EX_h2o_e', 'EX_h_e', 'EX_o2_e', 'EX_fe2_e', 'EX_fe3_e', 'EX_cobalt2_e', 'EX_ca2_e', 'EX_cl_e', 
       'EX_cu2_e','EX_k_e', 'EX_mg2_e', 'EX_mn2_e','EX_zn2_e', 'EX_mobd_e', 'EX_ni2_e', 'EX_na1_e']

for r in model.exchanges:
    r.lower_bound = 0
for r_id in uptake:
    model.reactions.get_by_id(r_id).lower_bound = -1000
    model.reactions.get_by_id(r_id).upper_bound = 1000
model.reactions.EX_gal_e.lower_bound = -10
print(model.optimize())

<Solution 0.490 at 0x16c1f3c70>


In [16]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
prot_pool,ER_pool_TG_,0.095,0,0.00%
ca2_e,EX_ca2_e,0.002553,0,0.00%
cl_e,EX_cl_e,0.002553,0,0.00%
cobalt2_e,EX_cobalt2_e,1.226E-05,0,0.00%
cu2_e,EX_cu2_e,0.0003478,0,0.00%
fe2_e,EX_fe2_e,0.007878,0,0.00%
gal_e,EX_gal_e,6.39,6,100.00%
k_e,EX_k_e,0.09574,0,0.00%
mg2_e,EX_mg2_e,0.004255,0,0.00%
mn2_e,EX_mn2_e,0.0003389,0,0.00%


# Do gene knockouts

In [18]:
genes = [g.id for g in model.genes]
essential_genes = []
pfba_sol_wt = cobra.flux_analysis.pfba(model)
sol_wt = model.optimize()
wt_growth = sol_wt.objective_value
for gene in genes:
    with model:
        g = model.genes.get_by_id(gene)
        g.knock_out()
        sol = model.slim_optimize()
        if sol < wt_growth*0.1:
            essential_genes.append(gene)
    

In [19]:
non_essential_genes = [g for g in genes if not g in essential_genes]

In [22]:
data = []
fluxes_dict = {}
shadow_price_dict = {}
fluxes_dict['WT'] = sol_wt.fluxes
for gene in non_essential_genes:
    with model:
        model.genes.get_by_id(gene).knock_out()
        sol = model.optimize()
        try:
            pfba_sol = cobra.flux_analysis.pfba(model)
        except:
            continue

        # Correlations between fluxes
        flux_pcorr, flux_ppval  = st.pearsonr(pfba_sol.fluxes, pfba_sol_wt.fluxes)
        flux_scorr, flux_spval  = st.spearmanr(pfba_sol.fluxes, pfba_sol_wt.fluxes)

        # Correlations between shadow prices
        sp_pcorr, sp_ppval  = st.pearsonr(pfba_sol.shadow_prices, pfba_sol_wt.shadow_prices)
        sp_scorr, sp_spval  = st.spearmanr(pfba_sol.shadow_prices, pfba_sol_wt.shadow_prices)
        data.append([gene, sol.objective_value, flux_pcorr, flux_ppval, flux_scorr, flux_spval, sp_pcorr, sp_ppval, sp_scorr, sp_spval])
        fluxes_dict[gene] = pfba_sol.fluxes
        shadow_price_dict[gene] = sol.shadow_prices

/Users/snorre/miniconda3/envs/reframed/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [23]:
shadow_price_dict['WT'] = sol_wt.shadow_prices

In [ ]:
df = pd.DataFrame(data, columns=['Gene','Growth rate', 'Flux pearson correlation', 'Flux pearson p-value', 'Flux spearman correlation', 'Flux spearman p-value',
                                        'Shadow price pearson correlation', 'Shadow price pearson p-value', 'Shadow price spearman correlation', 'Shadow price spearman p-value'])

In [ ]:
gene_name_dict = {g.id:g.name for g in modeli.genes}
gene_name_dict['WT'] = 'Wild-type'
df['Gene name'] = df.Gene.map(gene_name_dict)
df['Normalized growth rate'] = df['Growth rate']/wt_growth

## Read in metabolomics data

In [24]:
metabolomics_folder = Path('../../data/fuhrer_2017/')
neg_zscore = pd.read_csv(metabolomics_folder/'zscore_neg.tsv', sep = '\t', header=None)
pos_zscore = pd.read_csv(metabolomics_folder/'zscore_pos.tsv', sep = '\t', header=None)

In [26]:
zscores = pd.concat([neg_zscore, pos_zscore])

In [27]:
met_info = pd.read_excel(metabolomics_folder / 'metabolites.xlsx', sheet_name='Table EV1B')
met_info = met_info.iloc[1:,:4]

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [28]:
sample_info = pd.read_excel(metabolomics_folder / 'sample_id_zscore.xlsx', header = None)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.